In [2]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
pd.options.mode.chained_assignment=None

#PART 0 :-----------> parameters

#choose k, here for example i chosed the first 5 nearest neighbours
k=5

#the column to be dropped from the dataset

drop_cols=['room_type','city','zipcode','state','longitude','latitude','host_response_rate','host_acceptance_rate','host_response_rate','host_listings_count']






In [3]:
#PART 1 :-----------> data process

#read data
airbnb_list=pd.read_csv('paris_airbnb.csv')
print(airbnb_list.info())

#delete the non float and non interesting colomuns
airbnb_list=airbnb_list.drop(drop_cols,axis=1)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 19 columns):
host_response_rate      5000 non-null object
host_acceptance_rate    0 non-null float64
host_listings_count     7999 non-null float64
latitude                8000 non-null float64
longitude               8000 non-null float64
city                    7997 non-null object
zipcode                 7930 non-null object
state                   7977 non-null object
accommodates            8000 non-null int64
room_type               8000 non-null object
bedrooms                7976 non-null float64
bathrooms               7942 non-null float64
beds                    7986 non-null float64
price                   8000 non-null object
cleaning_fee            6250 non-null object
security_deposit        6320 non-null object
minimum_nights          8000 non-null int64
maximum_nights          8000 non-null int64
number_of_reviews       8000 non-null int64
dtypes: float64(7), int64(4), object(8

In [4]:
print(airbnb_list.info())
#check how many NULL values 
print(airbnb_list.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 10 columns):
accommodates         8000 non-null int64
bedrooms             7976 non-null float64
bathrooms            7942 non-null float64
beds                 7986 non-null float64
price                8000 non-null object
cleaning_fee         6250 non-null object
security_deposit     6320 non-null object
minimum_nights       8000 non-null int64
maximum_nights       8000 non-null int64
number_of_reviews    8000 non-null int64
dtypes: float64(3), int64(4), object(3)
memory usage: 625.1+ KB
None
accommodates            0
bedrooms               24
bathrooms              58
beds                   14
price                   0
cleaning_fee         1750
security_deposit     1680
minimum_nights          0
maximum_nights          0
number_of_reviews       0
dtype: int64


In [5]:
#clean the column price, remove , and $
airbnb_list['price']=airbnb_list['price'].astype(str).str.replace('$','')
airbnb_list['price']=airbnb_list['price'].astype(str).str.replace(',','')

#convert price column to float
airbnb_list['price']=airbnb_list['price'].astype('float')



In [6]:
#delete rows of bedrooms bathrooms and beds where are only little null values and 2 cols cleaning_fee and deposit_fee
airbnb_list=airbnb_list.drop(['cleaning_fee','security_deposit'],axis=1)
airbnb_list=airbnb_list.dropna(axis=0)
#check how many NULL values 
print(airbnb_list.isnull().sum())

accommodates         0
bedrooms             0
bathrooms            0
beds                 0
price                0
minimum_nights       0
maximum_nights       0
number_of_reviews    0
dtype: int64


In [7]:
#random the dataset
airbnb_list=airbnb_list.loc[np.random.permutation(len(airbnb_list))]

#noramalize data based on normal standard distribution
normalized_data=(airbnb_list-airbnb_list.mean())/(airbnb_list.std())
normalized_data=normalized_data.dropna(axis=0)
normalized_data['price']=airbnb_list['price']

#separate into train data and test data 75% 25%
sup=int(0.75*len(normalized_data))
train_data=normalized_data.iloc[0:sup]
test_data=normalized_data.iloc[sup:]
train_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5876 entries, 6998 to 5764
Data columns (total 8 columns):
accommodates         5876 non-null float64
bedrooms             5876 non-null float64
bathrooms            5876 non-null float64
beds                 5876 non-null float64
price                5876 non-null float64
minimum_nights       5876 non-null float64
maximum_nights       5876 non-null float64
number_of_reviews    5876 non-null float64
dtypes: float64(8)
memory usage: 413.2 KB


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


In [8]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5876 entries, 6998 to 5764
Data columns (total 8 columns):
accommodates         5876 non-null float64
bedrooms             5876 non-null float64
bathrooms            5876 non-null float64
beds                 5876 non-null float64
price                5876 non-null float64
minimum_nights       5876 non-null float64
maximum_nights       5876 non-null float64
number_of_reviews    5876 non-null float64
dtypes: float64(8)
memory usage: 413.2 KB


In [9]:
from sklearn.neighbors import KNeighborsRegressor
#init your knn model
knn=KNeighborsRegressor(n_neighbors=k,algorithm='brute')

#train features init
features = train_data.columns.tolist()
features.remove('price')
train_features=train_data[features]

#train target init
train_target = train_data ['price']

#luanch knn learning
knn.fit(train_features,train_target)


KNeighborsRegressor(algorithm='brute', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [10]:
predictions= knn.predict(test_data[features])


In [11]:
print(predictions)

[ 78.6  73.   91.6 ... 138.8 115.6  81.2]


In [12]:
#PART 5 :-----------> Evaluate
from sklearn.metrics import mean_squared_error

errorMSE=mean_squared_error(test_data['price'],predictions)

print('error mse= ', errorMSE, 'rmse= ',errorMSE**0.5)


error mse=  6706.612291985707 rmse=  81.89390875996644
